In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

benign_sites = pd.read_csv('/content/drive/MyDrive/majestic_million.csv')
phishing_sites = pd.read_csv('/content/drive/MyDrive/analysisdataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/majestic_million.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# prompt: shuffle the two dfs benign_sites and phishing_sites

benign_sites = benign_sites.sample(frac=1).reset_index(drop=True)


In [ ]:
benign_sites

In [ ]:
phishing_sites

In [ ]:
#@title extraction functions

import re
from bs4 import BeautifulSoup
import requests

def get_mobile_landing_url(url):
    headers = {'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 15_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1'}
    response = requests.get(url, headers=headers, timeout=1)
    if response.status_code != 200:
        return "offline"
    html_content = response.text
    if any(word in html_content for word in ["dangerous", "phishing", "suspected", "unsafe", "harmful"]):
        return "offline"
    soup = BeautifulSoup(response.text, 'html.parser')
    viewport_tag = soup.find('meta', attrs={'name': 'viewport'})
    if viewport_tag:
        return response.url

    return 'offline'

def extract_mobile_urls(urls):
    mobile_urls = []
    mobile_indicator_pattern = r"(m\.|mobile\.|mobi\.)"

    for url in urls:
        if re.search(mobile_indicator_pattern, url):
            mobile_urls.append(url)

    return mobile_urls

phishing_urls=[]
legitimate_urls=[]


In [ ]:
#@title url extraction benign

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import threading
from tqdm import tqdm


urls = benign_sites["Domain"].to_list()
processed_urls=[]
jobs=[]

def process_urls(url):
    try:
        url=get_mobile_landing_url(url)
        if url=='offline':
            return
        processed_urls.append(url)
    except:
        pass

max_urls = 5000

for i in tqdm(range(len(urls))):
    if len(processed_urls)>max_urls:
        break
    url=urls[i]
    if url.startswith("https://")==0:
        url="https://"+url
    t = threading.Thread(target=process_urls,args=(url,), name='process_urls')
    jobs.append(t)
    t.start()

for job in jobs:
    job.join()

legitimate_urls = processed_urls

In [ ]:
len(legitimate_urls)

In [ ]:
# prompt: convert legitimate_urls into dataframe with url as column name

legitimate_df = pd.DataFrame(legitimate_urls, columns=['url'])


In [ ]:
# prompt: export legitimate_df as csv with file name legitimate_urls_mobile

legitimate_df.to_csv('/content/drive/MyDrive/legitimate_urls_mobile_filtered.csv', index=False)


In [ ]:
#@title url extraction phishing

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import threading
import random
from tqdm import tqdm

urls = phishing_sites["Url"].to_list()
random.shuffle(urls)
processed_urls=[]
jobs=[]

def process_urls(url):
    try:
        url=get_mobile_landing_url(url)

        if url=="offline":
            return
        processed_urls.append(url)
    except:
        pass

prev=0
max_urls = 5000


for i in tqdm(range(len(urls))):
    if len(processed_urls)>max_urls:
        break
    url=urls[i]

    t = threading.Thread(target=process_urls,args=(url,), name='process_urls')
    jobs.append(t)
    t.start()

for job in jobs:
    job.join()

phishing_urls=processed_urls

In [ ]:
phishing_urls

In [ ]:
# prompt: convert phishing_urls into dataframe with url as the column name

import pandas as pd
phishing_df = pd.DataFrame(phishing_urls, columns=['url'])


In [ ]:
# prompt: export phishing_df as csv with name phishing_urls_mobile

phishing_df.to_csv('/content/drive/MyDrive/phishing_urls_mobile_filtered.csv', index=False)


In [ ]:
# prompt: add a column to legitimate url named as status with all entries as "benign"

legitimate_df['status'] = 'benign'


In [ ]:
legitimate_df

In [ ]:
# prompt: add a column to phishing_df named as status with all entries as "phishing"

phishing_df['status'] = 'phishing'


In [ ]:
# prompt: merge these two and shuffle the entries

import pandas as pd
merged_df = pd.concat([legitimate_df, phishing_df])
merged_df = merged_df.sample(frac=1).reset_index(drop=True)
merged_df


In [ ]:
# prompt: export merged_df as csv with name merged_urls_mobile

merged_df.to_csv('/content/drive/MyDrive/merged_urls_mobile_filtered.csv', index=False)
